In [1]:
import sys
import os
from ensemble_boxes import *
import torch
import random
import numpy as np
import pandas as pd
from glob import glob
from torch.utils.data import Dataset,DataLoader
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import cv2
import gc
from tqdm import tqdm
from matplotlib import pyplot as plt
from effdet import get_efficientdet_config, EfficientDet, DetBenchEval
from effdet.efficientdet import HeadNet
from sklearn.model_selection import StratifiedKFold
from skopt import gp_minimize, forest_minimize
from skopt.utils import use_named_args
from skopt.plots import plot_objective, plot_evaluations, plot_convergence, plot_regret
from skopt.space import Categorical, Integer, Real
import os
os.environ["CUDA_VISIBLE_DEVICES"]="3"
# SEED = 42

# def seed_everything(seed):
#     random.seed(seed)
#     os.environ['PYTHONHASHSEED'] = str(seed)
#     np.random.seed(seed)
#     torch.manual_seed(seed)
#     torch.cuda.manual_seed(seed)
#     torch.backends.cudnn.deterministic = True
#     torch.backends.cudnn.benchmark = True

# seed_everything(SEED)

In [2]:
USE_OPTIMIZE = True

In [3]:
marking = pd.read_csv('../data/train.csv')

bboxs = np.stack(marking['bbox'].apply(lambda x: np.fromstring(x[1:-1], sep=',')))
for i, column in enumerate(['x', 'y', 'w', 'h']):
    marking[column] = bboxs[:,i]
marking.drop(columns=['bbox'], inplace=True)

In [4]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
df_folds = marking[['image_id']].copy()
df_folds.loc[:, 'bbox_count'] = 1
df_folds = df_folds.groupby('image_id').count()
df_folds.loc[:, 'source'] = marking[['image_id', 'source']].groupby('image_id').min()['source']
df_folds.loc[:, 'stratify_group'] = np.char.add(
        df_folds['source'].values.astype(str),
        df_folds['bbox_count'].apply(lambda x: f'_{x // 15}').values.astype(str)
    )
df_folds.loc[:, 'fold'] = 0

for fold_number, (train_index, val_index) in enumerate(skf.split(X=df_folds.index, y=df_folds['stratify_group'])):
    df_folds.loc[df_folds.iloc[val_index].index, 'fold'] = fold_number

/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


In [5]:
def get_valid_transforms():
    return A.Compose(
        [
            A.Resize(height=1024, width=1024, p=1.0),
            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
            ToTensorV2(p=1.0),
        ], 
        p=1.0, 
        bbox_params=A.BboxParams(
            format='pascal_voc',
            min_area=0, 
            min_visibility=0,
            label_fields=['labels']
        )
    )

TRAIN_ROOT_PATH = '../data/train'

def collate_fn(batch):
    return tuple(zip(*batch))

class DatasetRetriever(Dataset):

    def __init__(self, marking, image_ids, transforms=None, test=False):
        super().__init__()

        self.image_ids = image_ids
        self.marking = marking
        self.transforms = transforms
        self.test = test

    def __getitem__(self, index: int):
        image_id = self.image_ids[index]
        image, boxes = self.load_image_and_boxes(index)
        labels = torch.ones((boxes.shape[0],), dtype=torch.int64)
        
        target = {}
        target['boxes'] = boxes
        target['labels'] = labels
        target['image_id'] = torch.tensor([index])

        if self.transforms:
            for i in range(10):
                sample = self.transforms(**{
                    'image': image,
                    'bboxes': target['boxes'],
                    'labels': labels
                })
                if len(sample['bboxes']) > 0:
                    image = sample['image']
                    target['boxes'] = torch.stack(tuple(map(torch.tensor, zip(*sample['bboxes'])))).permute(1, 0)
                    break

        return image, target, image_id

    def __len__(self) -> int:
        return self.image_ids.shape[0]

    def load_image_and_boxes(self, index):
        image_id = self.image_ids[index]
        image = cv2.imread(f'{TRAIN_ROOT_PATH}/{image_id}.jpg', cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        records = self.marking[self.marking['image_id'] == image_id]
        boxes = records[['x', 'y', 'w', 'h']].values
        boxes[:, 2] = boxes[:, 0] + boxes[:, 2]
        boxes[:, 3] = boxes[:, 1] + boxes[:, 3]
        return image, boxes

In [6]:
holdout_dataset = DatasetRetriever(
    image_ids=df_folds[df_folds.fold==2].index.values,
    marking=marking,
    transforms=get_valid_transforms(),
    test=True,
)

holdout_loader = DataLoader(
    holdout_dataset,
    batch_size=1,
    shuffle=False,
    num_workers=2,
    drop_last=False,
    collate_fn=collate_fn
)

In [7]:
def load_net(checkpoint_path):
    config = get_efficientdet_config('tf_efficientdet_d7')
    net = EfficientDet(config, pretrained_backbone=False)

    config.num_classes = 1
    config.image_size = 1024
    net.class_net = HeadNet(config, num_outputs=config.num_classes, norm_kwargs=dict(eps=.001, momentum=.01))

    checkpoint = torch.load(checkpoint_path)
    net = torch.nn.DataParallel(net)
    net.load_state_dict(checkpoint['ema_state_dict'])

    del checkpoint
    gc.collect()

    net = DetBenchEval(net, config)
    net.eval();
    return net.cuda()

if USE_OPTIMIZE:
    models = [
        load_net('./effdet7_fold2_coslr_step2/best-checkpoint-027epoch.bin'),
    ]

In [8]:
def make_tta_predictions(net,images, score_threshold=0.0):
    with torch.no_grad():
        x = torch.stack(images).float().cuda()
        predictions = []
        img_size = x.shape[-2:]  # height, width
        s = [0.83, 0.67]  # scales
        y = []
        all_x =[]
        for i, xi in enumerate((x,
                                x.flip(3), # only flip-lr
                                x.flip(2),
                                x.flip(2).flip(3),  # only flip-up
                                )):
            xp = xi
            all_x.append(xp)
            y.append(net(xp, torch.tensor([1]*xp.shape[0]).float().cuda()))
            
        y[1][..., 0] = img_size[1] - y[1][..., 0] - y[1][..., 2] # flip lr
        y[2][..., 1] = img_size[0] - y[2][..., 1] - y[2][..., 3]  # flip ud
        y[3][..., 1] = img_size[0] - y[3][..., 1] - y[3][..., 3]  # flip ud
        y[3][..., 0] = img_size[1] - y[3][..., 0] - y[3][..., 2] # flip lr
        
        
        y = np.array(y)
        boxes_all = []
        scores_all=[]
        for j in range(len(y)):
            boxes = y[j][0].cpu().numpy()[:,:4].copy() 
            scores = y[j][0].cpu().numpy()[:,4].copy()
            indexes = np.where(scores > score_threshold)[0]
            boxes = boxes[indexes]

            boxes[:, 2] = boxes[:, 2] + boxes[:, 0]
            boxes[:, 3] = boxes[:, 3] + boxes[:, 1]
            boxes = boxes.round().clip(min=0, max=1023)
            boxes_all.append(boxes/1024)
            scores_all.append(scores[indexes])
    return boxes_all,scores_all,y

In [9]:
if USE_OPTIMIZE:
    all_predictions = []
    for images, targets, image_ids in tqdm(holdout_loader, total=len(holdout_loader)):
        with torch.no_grad():
            fold_predictions = {}
            for fold_number in range(2,3):
                all_bbox,all_score,_ = make_tta_predictions(models[0],images)
            for i in range(1):
                image_predictions = {
                    'image_id': image_ids[i],
                    'gt_boxes': (targets[i]['boxes'].cpu().numpy()).clip(min=0, max=1023).astype(int),
                }
                for fold_number in range(2,3):
                    image_predictions[f'pred_boxes_fold{fold_number}'] = all_bbox
                    image_predictions[f'scores_fold{fold_number}'] = all_score

                all_predictions.append(image_predictions)

100%|██████████| 675/675 [15:05<00:00,  1.34s/it]


In [10]:
from test_utils import *

In [11]:
from tqdm.auto import tqdm

In [15]:
def calculate_final_score(
    all_predictions,
    iou_thr,
    skip_box_thr=0.0,
    th1=0.0,th2=0.0,th3=0.0,
#     score_threshold = [0.0,0.0,0.0]
    method='nms', # weighted_boxes_fusion, nms, soft_nms, non_maximum_weighted
    sigma=0.5,x1=1.,x2=1.,x3=1.,
):
    final_scores = []
    weights=[th1,th2,th3]
    score_threshold = [th1,th2,th3]
    for i in tqdm(range(len(all_predictions))):
        gt_boxes = all_predictions[i]['gt_boxes'].copy().astype(float)/1024
        image_id = all_predictions[i]['image_id']
        folds_boxes, folds_scores, folds_labels = [], [], []
        for fold_number in range(2,3):
            folds_boxes = all_predictions[i][f'pred_boxes_fold{fold_number}'].copy()[0:3]
            folds_scores = all_predictions[i][f'scores_fold{fold_number}'].copy()[0:3]
#             folds_labels = [np.ones([row.shape[0]]) for row in folds_scores]
        folds_boxes_new = []
        folds_scores_new = []
        for bb in range(len(folds_boxes)):
            s = folds_scores[bb].copy()
            b = folds_boxes[bb].copy()
            indexes = np.where(s > score_threshold[bb])[0]
            new_bbox = b[indexes]
            new_score = s[indexes]
            folds_boxes_new.append(new_bbox)
            folds_scores_new.append(new_score)
        folds_boxes = folds_boxes_new
        folds_scores = folds_scores_new
        folds_labels = [np.ones([row.shape[0]]) for row in folds_scores_new]
        if method == 'weighted_boxes_fusion':
            boxes, scores, labels = weighted_boxes_fusion(folds_boxes, folds_scores, folds_labels, weights=weights, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
        elif method == 'nms':
            try:
                boxes, scores, labels = nms(folds_boxes, folds_scores, folds_labels, weights=weights, iou_thr=iou_thr)
            except:
                boxes, scores, labels = weighted_boxes_fusion(folds_boxes, folds_scores, folds_labels, weights=weights, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
        elif method == 'soft_nms':
            boxes, scores, labels = soft_nms(folds_boxes, folds_scores, folds_labels, weights=weights, iou_thr=iou_thr, thresh=skip_box_thr, sigma=sigma)
        elif method == 'non_maximum_weighted':
            boxes, scores, labels = non_maximum_weighted(folds_boxes, folds_scores, folds_labels, weights=weights, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
        else:
            raise
        image_precision = calculate_image_precision(gt_boxes, boxes, thresholds=iou_thresholds, form='pascal_voc')
        final_scores.append(image_precision)

    return np.mean(final_scores)

In [13]:
%%time
#{'iou_thr': 0.43312889428044965, 'score_threshold': 0.4024215973594928, 'skip_box_thr': 0.393358169307333}
# standart params as here https://github.com/ZFTurbo/Weighted-Boxes-Fusion
if USE_OPTIMIZE:
    print('[WBF]: ', calculate_final_score(
        all_predictions, 
        iou_thr=0.43312889428044965,
        skip_box_thr=0.393358169307333,
        score_threshold = 0.4024215973594928,
        method='weighted_boxes_fusion',weights=[0.6,0.2,0.2]
    ))


[WBF]:  0.910118150102896
CPU times: user 52.9 s, sys: 873 ms, total: 53.8 s
Wall time: 52.6 s


In [16]:
def log(text):
    with open('opt.log', 'a+') as logger:
        logger.write(f'{text}\n')

def optimize(space, all_predictions, method, n_calls=10):
    @use_named_args(space)
    def score(**params):
        print('-'*5 + f'{method}' + '-'*5)
        print(params)
        final_score = calculate_final_score(all_predictions, method=method, **params)
        print(f'final_score = {final_score}')
        print('-'*10)
        return -final_score

    return gp_minimize(func=score, dimensions=space, n_calls=n_calls)

# WBF

In [19]:
%%time

space = [
    Real(0.3, 0.75, name='iou_thr'),
#     Real(0.2, 0.5, name='score_threshold'),
    Real(0.3, 0.6, name='skip_box_thr'),
    Real(0.4, 0.9, name='x1'),
    Real(0.1, 0.5, name='x2'),
    Real(0.1, 0.5, name='x3'),
    Real(0.25, 0.55, name='th1'),
    Real(0.25, 0.55, name='th2'),
    Real(0.25, 0.55, name='th3'),
]

if USE_OPTIMIZE:
    opt_result = optimize(
        space, 
        all_predictions,
        method='weighted_boxes_fusion',
        n_calls=250,
    )

-----weighted_boxes_fusion-----
{'iou_thr': 0.7067305971843766, 'skip_box_thr': 0.35498303839764217, 'x1': 0.4232392179059292, 'x2': 0.16460073494653982, 'x3': 0.24252242047241415, 'th1': 0.37643231085301543, 'th2': 0.32732057825586053, 'th3': 0.35771949103535405}



final_score = 0.8924832045504647
----------
-----weighted_boxes_fusion-----
{'iou_thr': 0.3325461053370589, 'skip_box_thr': 0.5152069156513917, 'x1': 0.415608184184478, 'x2': 0.17936017241779587, 'x3': 0.2020145710701497, 'th1': 0.4413476769432363, 'th2': 0.2765997120210041, 'th3': 0.2960576806286025}



final_score = 0.8726338641997865
----------
-----weighted_boxes_fusion-----
{'iou_thr': 0.3119971916148788, 'skip_box_thr': 0.3408336292553492, 'x1': 0.5557104479302463, 'x2': 0.1889576265516489, 'x3': 0.40851620998176597, 'th1': 0.3834306027593464, 'th2': 0.2929899272920093, 'th3': 0.2968726008695294}



final_score = 0.9053540619821895
----------
-----weighted_boxes_fusion-----
{'iou_thr': 0.5690141150460041, 'skip_box_thr': 0.30058191271176304, 'x1': 0.5510058409625869, 'x2': 0.2700899495530412, 'x3': 0.3197930006437815, 'th1': 0.42054114826897726, 'th2': 0.3446835335320742, 'th3': 0.25198495082883826}



final_score = 0.8835858287461064
----------
-----weighted_boxes_fusion-----
{'iou_thr': 0.3416047302183488, 'skip_box_thr': 0.35014337582198796, 'x1': 0.7359547851030518, 'x2': 0.20009866818574928, 'x3': 0.1285942271050584, 'th1': 0.29501457357738403, 'th2': 0.4102744933279364, 'th3': 0.474117700459758}


...


final_score = 0.910315316088711
----------
-----weighted_boxes_fusion-----
{'iou_thr': 0.4236807224044294, 'skip_box_thr': 0.33746892069988316, 'x1': 0.4, 'x2': 0.5, 'x3': 0.259780245175151, 'th1': 0.55, 'th2': 0.402493171472236, 'th3': 0.38299100055653834}



final_score = 0.910346802922298
----------
-----weighted_boxes_fusion-----
{'iou_thr': 0.42568514351291464, 'skip_box_thr': 0.33697731347475596, 'x1': 0.5518075777191463, 'x2': 0.5, 'x3': 0.21295548936884157, 'th1': 0.55, 'th2': 0.40574530757669497, 'th3': 0.38190491497375084}



final_score = 0.9102494234090172
----------
-----weighted_boxes_fusion-----
{'iou_thr': 0.4233827339179177, 'skip_box_thr': 0.33732219219967086, 'x1': 0.5173511759355887, 'x2': 0.5, 'x3': 0.431927198373648, 'th1': 0.55, 'th2': 0.40182434951201873, 'th3': 0.38266415011938976}



final_score = 0.9102300612378069
----------
-----weighted_boxes_fusion-----
{'iou_thr': 0.4226674720865691, 'skip_box_thr': 0.3373541552089461, 'x1': 0.6857272781991102, 'x2': 0.5, 'x3': 0.15894362140197077, 'th1': 0.55, 'th2': 0.4024108838702133, 'th3': 0.38266484519685995}



final_score = 0.9103591106273516
----------
CPU times: user 4h 16min 23s, sys: 1h 4min 21s, total: 5h 20min 45s
Wall time: 3h 46min 59s


In [16]:
if USE_OPTIMIZE:
    best_final_score = -opt_result.fun
    best_iou_thr = opt_result.x[0]
    best_skip_box_thr = opt_result.x[1]
else:
    # calculated early for fast inference in submission [see version 1 with n_calls=300]
    best_final_score = 0.6953
    best_iou_thr = 0.44
    best_skip_box_thr = 0.43

print('-'*13 + 'WBF' + '-'*14)
print(f'[Best Iou Thr]: {best_iou_thr:.3f}')
print(f'[Best Skip Box Thr]: {best_skip_box_thr:.3f}')
print(f'[Best Score]: {best_final_score:.4f}')
print('-'*30)
print(opt_result.x)

-------------WBF--------------
[Best Iou Thr]: 0.414
[Best Skip Box Thr]: 0.382
[Best Score]: 0.9034
------------------------------
[0.41437273872034813, 0.38220680280489394, 0.3945765571800917]


In [29]:
-------------WBF--------------
[Best Iou Thr]: 0.441
[Best Skip Box Thr]: 0.367
[Best Score]: 0.9065
------------------------------

SyntaxError: invalid syntax (<ipython-input-29-a2df3b8a11ab>, line 1)

In [ ]:
-------------nms--------------
[Best Iou Thr]: 0.408
[Best Skip Box Thr]: 0.391
[Best Score]: 0.9076
------------------------------